In [6]:
#coding:utf-8
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
#read data
train = pd.read_csv("data/train.csv",dtype={"Age": np.float64},)
test = pd.read_csv("data/test.csv",dtype={"Age": np.float64},)

In [8]:
def harmonize_data(titanic):#填充空数据 和 把string数据转成integer表示
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    titanic.Age[titanic.Age < 15] = 0
    titanic.Age[titanic.Age >= 15] = 1

    titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
    titanic.loc[titanic["Sex"] == "female", "Sex"] = 1

    titanic["Embarked"] = titanic["Embarked"].fillna("S")

    titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 1
    titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 2
    titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 3

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic

In [9]:
train_data = harmonize_data(train)
test_data =harmonize_data(test)

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


## use split cv

In [61]:
train, valid = train_test_split(train_data, test_size = 0.3, random_state=1)
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

x_train = train[predictors]
y_train = train["Survived"]

x_valid = valid[predictors]
y_valid = valid["Survived"]

In [ ]:
rf = RandomForestClassifier(
    #random_state=1,
    n_estimators=500,
    min_samples_split=12,
    min_samples_leaf=1,
    oob_score=True
)

rf.fit(x_train,y_train)
y_pre = rf.predict(x_valid)
print accuracy_score(y_valid, y_pre)
predict_data = rf.predict(test_data[predictors])

## use kfold cv

In [59]:
rf2 = RandomForestClassifier()
scores = cross_val_score(rf2, train_data[predictors], train_data["Survived"], cv=10, scoring='accuracy')
print scores.mean()

0.819358472364


In [57]:
predict_dataframe = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predict_data
})

In [58]:
predict_dataframe.to_csv('data/randomforest.csv',index=False,encoding="utf-8")